In [4]:
##%overwritefile
##%file:src/compile_with_tsc.py
##%noruncode
    def compile_with_tsc(self, source_filename, binary_filename, cflags=None, ldflags=None,env=None,magics=None):
        outfile=binary_filename
        orig_cflags=cflags
        orig_ldflags=ldflags
        index=0
        for s in cflags:
            if s.startswith('--outFile'):
                if(len(s)>9):
                    outfile=s[9:]
                    del cflags[index]
                else:
                    outfile=cflags[cflags.index('--outFile')+1]
                    if outfile.startswith('-'):
                        outfile=binary_filename
                    del cflags[cflags.index('--outFile')+1]
                    del cflags[cflags.index('--outFile')]
            binary_filename=outfile
            index+=1
        args=[]
        if magics!=None and len(self.mymagics.addkey2dict(magics,'ccompiler'))>0:
            args = magics['ccompiler'] + orig_cflags +[source_filename] + orig_ldflags
        else:
            tsccmd=['tsc']
            if self.mymagics.sys=="Windows":
                tsccmd=['cmd','/c','tsc']
            args = tsccmd+[source_filename] + ['--outFile', binary_filename]+ cflags  + ldflags
        # self._log(''.join((' '+ str(s) for s in args))+"\n")
        return self.mymagics.create_jupyter_subprocess(args,env=env,magics=magics),binary_filename,args


[MyPythonKernel215222] Info:src/compile_with_tsc.py
[MyPythonKernel215222] Info:file h:\Jupyter\Myjupyter-kernel\kernel\TypeScript\src/compile_with_tsc.py created successfully


In [5]:
##%overwritefile
##%file:src/c_exec_tsc_.py
##%noruncode
    def _exec_tsc_(self,source_filename,magics):
        self.mymagics._logln('Generating executable file')
        with self.mymagics.new_temp_file(suffix='.js') as binary_file:
            magics['status']='compiling'
            p,outfile,tsccmd = self.compile_with_tsc(
                source_filename, 
                binary_file.name,
                self.mymagics.get_magicsSvalue(magics,'cflags'),
                self.mymagics.get_magicsSvalue(magics,'ldflags'),
                self.mymagics.get_magicsbykey(magics,'env'),
                magics=magics)
            returncode=p.wait_end(magics)
            p.write_contents()
            magics['status']=''
            binary_file.name=os.path.join(os.path.abspath(''),outfile)
            if returncode != 0:  # Compilation failed
                self.mymagics._logln(' '.join((str(s) for s in tsccmd))+"\n",3)
                self.mymagics._logln("TSC compiler exited with code {}, the executable will not be executed".format(returncode),3)

                # delete source files before exit
                os.remove(source_filename)
                os.remove(binary_file.name)
        return p.returncode,binary_file.name


[MyPythonKernel215222] Info:src/c_exec_tsc_.py
[MyPythonKernel215222] Info:file h:\Jupyter\Myjupyter-kernel\kernel\TypeScript\src/c_exec_tsc_.py created successfully
